In [37]:
import pandas as pd 
import spacy
import numpy as np

In [38]:
nlp = spacy.load("en_core_web_sm") 

In [61]:
df = pd.read_json('../data/News_Category_Dataset_v3.json', lines=True)

In [65]:
counts = df.category.value_counts()

In [66]:
labels = df.category.unique()
col = df.columns

In [67]:
col

Index(['link', 'headline', 'category', 'short_description', 'authors', 'date'], dtype='object')

In [68]:
df1 = pd.DataFrame(columns=col)

In [69]:
i = 0
for label in labels:
    if counts[label]>5000 and i<5:
        df2 = df[df.category==label].sample(250, random_state=2022)
        df1 = pd.concat([df1, df2], axis=0)
        i+=1

In [70]:
df1.category.value_counts()

COMEDY           250
PARENTING        250
SPORTS           250
ENTERTAINMENT    250
POLITICS         250
Name: category, dtype: int64

In [71]:
target = {}
i = 0
for label in labels:
    target[label] = i
    i+=1

In [72]:
df1["category_num"] = df['category'].map(target)

In [74]:
def preprocess(text):
    doc = nlp(text)
    temp = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        temp.append(token.text)
    return ' '.join(temp)

In [56]:
df1['short_description'] = df1['short_description'].astype(str)

In [57]:
df1['text'] = df1['short_description'].apply(lambda text: preprocess(text))

In [58]:
df1['vector'] = df1[ 'text'].apply(lambda text: nlp(text).vector)

In [59]:
df1.head(10)

,link,headline,category,short_description,authors,date,category_num,text,vector
15312,https://www.huffingtonpost.com/entry/trevor-no...,"Trevor Noah Calls Out Hillary Clinton, And He'...",COMEDY,"The ""Daily Show"" host condemned Clinton for no...",Ron Dicker,2018-01-30,1,Daily host condemned Clinton firing adviser ac...,"[-0.30942327, 0.4471988, 0.06940637, -0.280289..."
126777,https://www.huffingtonpost.com/entry/cat-ninja...,What Sorcery Is This?,COMEDY,,Carol Hartsell,2014-07-07,1,,[]
47238,https://www.huffingtonpost.com/entry/hi-death-...,"Hi, Death? I'd Like To Speak To Your Manager, ...",COMEDY,The service has been quite terrible lately.,Andy McDonald,2016-12-30,1,service terrible lately,"[0.11467292, 0.6868064, 0.061303835, 0.0076178..."
119663,https://www.huffingtonpost.com/entry/new-again...,...New Again?,COMEDY,,"Marcia Liss, Contributor(Almost) Famous Cartoo...",2014-09-26,1,,[]
30724,https://www.huffingtonpost.com/entry/there-was...,"There Was No Audio, So We Captioned The Trump ...",COMEDY,Shame recognize shame.,Andy McDonald,2017-07-07,1,Shame recognize shame,"[0.056958873, -0.08312402, 0.5640574, -0.54336..."
51307,https://www.huffingtonpost.com/entry/kate-mcki...,Fake Ruth Bader Ginsburg Won't Quit Under Pres...,COMEDY,"""I’m eating an apple a day to keep Ben Carson ...",Lee Moran,2016-11-13,1,eating apple day Ben Carson away,"[0.48893794, 0.08664507, -0.0056601763, -0.405..."
123664,https://www.huffingtonpost.com/entry/10-of-the...,10 Of The Most Creative Acts Of Vandalism Ever,COMEDY,,,2014-08-11,1,,[]
13885,https://www.huffingtonpost.com/entry/jimmy-kim...,Jimmy Kimmel's Fake Olympics Takes Another Tur...,COMEDY,Ouch!,Lee Moran,2018-02-21,1,Ouch,"[-0.033032298, 0.5720664, 0.74516773, -1.22504..."
85444,https://www.huffingtonpost.com/entry/what-the-...,What The Benghazi Hearings Would Look Like If ...,COMEDY,"""Hey, c'mon guys. Let's get outta here and pol...",John Trowbridge,2015-10-22,1,Hey c'm guys Let outta polish Miller High Life,"[0.7651166, -0.27829581, -0.15790081, -0.28048..."
75876,https://www.huffingtonpost.com/entry/star-wars...,Brilliant Human Creates 'Star Wars' 'Making A ...,COMEDY,"That'll do, Internet. That'll do.",Andy McDonald,2016-02-08,1,Internet,"[0.7534405, 0.8921896, 0.24037391, -1.3566835,..."


In [52]:
x = df1.vector

In [53]:
x = np.stack(x)

ValueError: all input arrays must have the same shape

In [ ]:
y = df1['category_num']

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, stratify=y, random_state=2022)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

model = Pipeline([
    ("vec", CountVectorizer(ngram_range=(1,1))),
    ("model", MultinomialNB())
])

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print(classification_report(y_test, y_pred))